In [7]:
import PyPDF2
import pdfplumber as pr
import pandas as pd
from openpyxl import load_workbook
import re
import os

In [6]:
PDF_files  = []
input_path = r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input'
for x in os.listdir(input_path):
    if '.pdf' in x:
        PDF_files.append(x)

GFI  = pd.read_excel(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input/202111 政府基金運用資訊 v2.0.xlsx',sheet_name = 'Government Fund Info (Raw Data)',skiprows = 2)
# 
#把index處理一下
new_index = []
old_index = GFI.set_index('Unnamed: 0').index
for i in GFI.set_index('Unnamed: 0').index:
    #只抓前面的中文部分
    if re.match('[\u4e00-\u9fa5]+',i):
        new_index.append(re.match('[\u4e00-\u9fa5]+',i)[0])

In [ ]:
def convert_勞退_dict(x):
    勞退_dict = {
        '公債、公司債、金融債券及特別股':'公債',
        '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
        '國內委託經營':'國內委託',
        '國外委託經營':'國外委託',
        '合            計':'金額總計'
    }
    if x in 勞退_dict:
        result = 勞退_dict[x]
    else:
        result = x
    return result
def convert_勞保_dict(x):
    勞保_dict = {
        '公債、公司債、金融債券及特別股':'公債',
        '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
        '國內委託經營':'國內委託',
        '國外委託經營':'國外委託',
        '合    計':'金額總計'
    }
    if x in 勞保_dict:
        result = 勞保_dict[x]
    else:
        result = x
    return result
def convert_國保_dict(x):
    國保_dict = {
        '公債、公司債、金融債券及特別股':'公債',
        '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
        '合           計':'金額總計'
    }
    if x in 國保_dict:
        result = 國保_dict[x]
    else:
        result = x
    return result
def convert_退撫_dict(x):
    退撫_dict = {
        '自行運用小計':'自行運用',
        '國內委託經營':'國內委託',
        '國外委託經營':'國外委託',
        '委託經營小計':'委託經營',
        '合                 計':'金額總計'
    }
    if x in 退撫_dict:
        result = 退撫_dict[x]
    else:
        result = x
    return result

In [10]:
for pdf_file in PDF_files:
    if '國民年金' in pdf_file:
        國保_pdf = pr.open(input_path+'/'+pdf_file)
    elif '勞工退休' in pdf_file:
        勞退_pdf = pr.open(input_path+'/'+pdf_file)
    elif '基金運用' in pdf_file:
        退撫_pdf = pr.open(input_path+'/'+pdf_file)
    elif '勞工保險' in pdf_file:
        勞保_pdf = pr.open(input_path+'/'+pdf_file)

In [21]:
勞退_pdf.pages[2].extract_tables()

[[['項          目', '餘       額', '占基金運用比率\n（％）'],
  ['自行運用', '1,306,099,568,604', '40.94'],
  ['轉存金融機構', '619,950,926,538', '19.43'],
  ['短期票券', '53,056,994,167', '1.66'],
  ['公債、公司債、金融債券及特別股', '202,917,481,927', '6.36'],
  ['股票及受益憑證投資（含期貨）', '177,709,826,394', '5.57'],
  ['國外投資', '252,464,339,578', '7.92'],
  ['固定收益', '152,043,571,812', '4.77'],
  ['權益證券', '53,017,947,751', '1.66'],
  ['另類投資', '47,402,820,015', '1.49'],
  ['委託經營', '1,884,537,399,059', '59.06'],
  ['國內委託經營', '468,246,576,073', '14.68'],
  ['國外委託經營', '1,416,290,822,986', '44.38'],
  ['固定收益', '357,161,499,961', '11.19'],
  ['權益證券', '704,226,264,484', '22.07'],
  ['另類投資', '354,903,058,541', '11.12'],
  ['合            計', '3,190,636,967,663', '100.00']]]

In [ ]:
#讀取指定文件內的所有圖表
def get_all_table(file):
    tables_dict = {}
    ps          = file.pages
    print(len(ps))
    i           = 0
    for p in range(len(ps)):
        #進去每一頁
        pg      = ps[p]
        #找到那頁的有表格
        tables  = pg.extract_tables()
        #一頁只有一個表格的情況
        if len(tables) == 1:
            table = tables
            tables_dict['表格' + str(i)] = table
            i += 1
        #一頁有超過一個表格的情況
        elif len(tables) > 1:
            for g in range(len(tables)):
                table = tables[g]
                tables_dict['表格' + str(i)] = table
                i += 1
    return tables_dict

In [11]:
#按照我們設定的行數來尋找目標table，這要人工去看每個目標的行數
def get_target_table(tables_dict,target_len):
    i = 0
    #用來放目標table的
    target_table_df = pd.DataFrame({})
    #用表格的長度來決定要抓什麼表格
    for table in list(tables_dict.values()):
        if len(table) == target_len:
            print(i)
            Table = pd.DataFrame(table[0][1:],columns = table[0][0])
            target_table_df = pd.concat([target_table_df,Table],axis=1)
        
        #這邊是發現有些表格會是三維的，所以做特殊處理
        elif len(table[0]) == target_len:
            print(i)
            Table = pd.DataFrame(table[0][1:],columns = table[0][0])
            #因為項目我已經有了，所以要拿掉
            target_table_df = pd.concat([target_table_df,Table],axis=1)
        i += 1
    return target_table_df

In [ ]:
def handle_勞退_process(DataFrame):
    DataFrame.columns = ['項          目','勞退新制\nLPF','占基金運用比率（％）新','項          目','勞退舊制\nLRF','占基金運用比率（％）舊']
    #去掉蘭重複的
    新舊制勞對_df       = DataFrame.loc[:,]